In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [3]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_175_2','COVID-19'],
    subset='training'
) 

Found 350 images belonging to 2 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_20_2','COVID-19'],
    subset='training'
)

Found 40 images belonging to 2 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal_20_2','COVID-19'],
        #subset='validation'
)

Found 40 images belonging to 2 classes.


In [7]:
new_input = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.DenseNet169(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# se acopla el modelo
model_1 = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model_1.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_1.summary()

___________
conv5_block21_0_relu (Activatio (None, 7, 7, 1280)   0           conv5_block21_0_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_1_conv (Conv2D)   (None, 7, 7, 128)    163840      conv5_block21_0_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_bn (BatchNormal (None, 7, 7, 128)    512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 7, 7, 128)    0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block21_1_relu[0][0]       
_________________________________________________________________________________________________

In [8]:
#inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

base_model1 = keras.applications.VGG19(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model1.trainable = False

x1 = base_model1.output
x1 = keras.layers.Flatten()(x1)
x1 = keras.layers.Dropout(0.4)(x1)
x1 = keras.layers.Dense(2048, activation='relu')(x1)
x1 = keras.layers.BatchNormalization()(x1)
x1 = keras.layers.Dropout(0.4)(x1)
x1 = keras.layers.Dense(2048, activation='relu')(x1)
x1 = keras.layers.BatchNormalization()(x1)
x1 = keras.layers.Dropout(0.2)(x1)
outputs = keras.layers.Dense(1, activation='sigmoid')(x1)

# se acopla el modelo
model_2 = keras.Model(base_model1.input, outputs)

# congelar capas
for layer in base_model1.layers:
    layer.trainable = False

# compilar el modelo.
model_2.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [9]:
base_model2 = keras.applications.InceptionV3(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model2.trainable = False

x2 = base_model2.output
x2 = keras.layers.Flatten()(x2)
x2 = keras.layers.Dropout(0.4)(x2)
x2 = keras.layers.Dense(2048, activation='relu')(x2)
x2 = keras.layers.BatchNormalization()(x2)
x2 = keras.layers.Dropout(0.4)(x2)
x2 = keras.layers.Dense(2048, activation='relu')(x2)
x2 = keras.layers.BatchNormalization()(x2)
x2 = keras.layers.Dropout(0.2)(x2)
outputs = keras.layers.Dense(1, activation='sigmoid')(x2)

# se acopla el modelo
model_3 = keras.Model(base_model2.input, outputs)

# congelar capas
for layer in base_model2.layers:
    layer.trainable = False

# compilar el modelo.
model_3.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_3.summary()

[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 12, 12, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]              
                                                                 activation_69[0][0]              
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 12, 12, 192)  576         conv2d_72[0][0]                  
_________________________________________________________________________________________________

In [10]:
base_model3 = keras.applications.ResNet50(
    weights = 'imagenet',
    #input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,input_tensor=new_input)

base_model3.trainable = False

x3 = base_model2.output
x3 = keras.layers.Flatten()(x3)
x3 = keras.layers.Dropout(0.4)(x3)
x3 = keras.layers.Dense(2048, activation='relu')(x3)
x3 = keras.layers.BatchNormalization()(x3)
x3 = keras.layers.Dropout(0.4)(x3)
x3 = keras.layers.Dense(2048, activation='relu')(x3)
x3 = keras.layers.BatchNormalization()(x3)
x3 = keras.layers.Dropout(0.2)(x3)
outputs = keras.layers.Dense(1, activation='sigmoid')(x3)

# se acopla el modelo
model_4 = keras.Model(base_model2.input, outputs)

# congelar capas
for layer in base_model3.layers:
    layer.trainable = False

# compilar el modelo.
model_4.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model_4.summary()

[0][0]     
__________________________________________________________________________________________________
mixed7 (Concatenate)            (None, 12, 12, 768)  0           activation_60[0][0]              
                                                                 activation_63[0][0]              
                                                                 activation_68[0][0]              
                                                                 activation_69[0][0]              
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 12, 12, 192)  147456      mixed7[0][0]                     
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 12, 12, 192)  576         conv2d_72[0][0]                  
_________________________________________________________________________________________________

In [11]:
mergedOutput = tf.keras.layers.Concatenate()([model_1.output,model_2.output,model_3.output,model_4.output])

hidden = tf.keras.layers.Dense(10, activation='relu')(mergedOutput)
output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)
model = tf.keras.Model(inputs=new_input, outputs=output)
                # plot graph of ensemble
            #plot_model(model, show_shapes=True, to_file='model_graph.png')
                # compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


           
__________________________________________________________________________________________________
mixed9 (Concatenate)            (None, 5, 5, 2048)   0           activation_76[0][0]              
                                                                 mixed9_0[0][0]                   
                                                                 concatenate[0][0]                
                                                                 activation_84[0][0]              
__________________________________________________________________________________________________
conv5_block31_2_conv (Conv2D)   (None, 7, 7, 32)     36864       conv5_block31_1_relu[0][0]       
__________________________________________________________________________________________________
block3_pool (MaxPooling2D)      (None, 28, 28, 256)  0           block3_conv4[0][0]               
_________________________________________________________________________________________________

In [12]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 5)

Epoch 1/5
10/10 [==============================] - 330s 31s/step - loss: 0.6231 - accuracy: 0.6304 - val_loss: 0.6902 - val_accuracy: 0.4375
Epoch 2/5
10/10 [==============================] - 367s 37s/step - loss: 0.5470 - accuracy: 0.7888 - val_loss: 0.7239 - val_accuracy: 0.3125
Epoch 3/5
10/10 [==============================] - 316s 32s/step - loss: 0.5107 - accuracy: 0.8517 - val_loss: 0.5739 - val_accuracy: 0.9062
Epoch 4/5
10/10 [==============================] - 612s 64s/step - loss: 0.4792 - accuracy: 0.8865 - val_loss: 0.5385 - val_accuracy: 0.9062
Epoch 5/5
10/10 [==============================] - 253s 25s/step - loss: 0.4834 - accuracy: 0.8724 - val_loss: 0.5499 - val_accuracy: 0.9062


In [13]:
results = model.evaluate(test_gen, batch_size=10)

2/2 [==============================] - 15s 3s/step - loss: 0.5668 - accuracy: 0.8750


In [14]:
model.save('F:/X_ray_Models/e2_binary_model_4')

history.history

INFO:tensorflow:Assets written to: F:/X_ray_Models/e2_binary_model_4\assets


{'loss': [0.5978471636772156,
  0.5477563738822937,
  0.49613797664642334,
  0.4789096713066101,
  0.4761456251144409],
 'accuracy': [0.6792452931404114,
  0.7798742055892944,
  0.8773584961891174,
  0.8899371027946472,
  0.8773584961891174],
 'val_loss': [0.6902251243591309,
  0.7239302396774292,
  0.5739426016807556,
  0.5384764671325684,
  0.549883246421814],
 'val_accuracy': [0.4375, 0.3125, 0.90625, 0.90625, 0.90625]}

In [15]:
y_pred = model.predict(test_gen, verbose=1)

#y_pred = np.argmax(y_pred, axis = 1)


for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

2/2 [==============================] - 35s 3s/step


In [16]:
y_true=test_gen.labels

In [17]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.70      0.68        20
           1       0.68      0.65      0.67        20

    accuracy                           0.68        40
   macro avg       0.68      0.68      0.67        40
weighted avg       0.68      0.68      0.67        40



In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[14,  6],
       [ 7, 13]], dtype=int64)